In [ ]:
import numpy as np

In [ ]:
images_url = "./CUB_200_2011/images.txt"
train_test_split_url = "./CUB_200_2011/train_test_split.txt"
classes_url = "./CUB_200_2011/classes.txt"
image_class_labels_url = "./CUB_200_2011/image_class_labels.txt"

Loading .txt file

In [ ]:
image = np.genfromtxt(images_url , delimiter=' ', dtype=str) #<image_id> <image_name>
image = {int(row[0]): "CUB_200_2011/images/"+row[1] for row in image}

train_test_split = np.genfromtxt(train_test_split_url, delimiter=" ", dtype=str) #<image_id> <is_training_image>
train_test_split = {int(row[0]): int(row[1]) for row in train_test_split}

classes = np.genfromtxt(classes_url, delimiter=" ", dtype=str) #<class_id> <class_name>
classes = {int(row[0]): row[1] for row in classes}

image_class_labels = np.genfromtxt(image_class_labels_url, delimiter=" ", dtype=str).astype(int) #<image_id> <class_id>
image_class_labels = {int(row[0]): int(row[1]) for row in image_class_labels}

<table>
    <tr>
        <td>image</td>
        <td>&lt;image_id&gt; &lt;image_name&gt;</td>
    </tr>
    <tr>
        <td>train_test_split</td>
        <td>&lt;image_id&gt; &lt;is_training_image&gt;</td>
    </tr>
    <tr>
        <td>classes</td>
        <td>&lt;class_id&gt; &lt;class_name&gt;</td>
    </tr>
    <tr>
        <td>image_class_labels</td>
        <td>&lt;image_id&gt; &lt;class_id&gt;</td>
    </tr>
</table>

In [ ]:
row = 5
print(row, image[row])
print(row, train_test_split[row])
print(row, classes[row])
print(row, image_class_labels[row])

In [ ]:
# image_train = {key: value for key, value in  train_test_split.items() if value == 1}
# image_test = {key: value for key, value in  train_test_split.items() if value == 0}
image_train = np.array([key for key, value in train_test_split.items() if value == 1])
image_test = np.array([key for key, value in train_test_split.items() if value == 0])

n_train = len(image_train)
n_test = len(image_test)
n_classes = len(classes)

print("Number of Images:", len(image))
print(f"n_train: {n_train}")
print(f"n_test: {n_test}")
print(f"n_classes: {n_classes}")

<table>
    <tr>
        <td>image_train</td>
        <td>list of &lt;image_id&gt;</td>
        <td>&lt;is_training_image&gt; == 1</td>
    </tr>
    <tr>
        <td>image_test</td>
        <td>list of &lt;image_id&gt;</td>
        <td>&lt;is_training_image&gt; == 0</td>
    </tr>
</table>

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def showImage(image):
    plt.imshow(plt.imread(image))
    plt.axis(False)
    plt.show()

In [ ]:
showImage(image[10])

## Model
### EfficientNet_B1_V2

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

In [ ]:
# hyper parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_out_ftrs = 200 # number of classes for out classification is 200
image_input_size = 244 # 244x244 image; imagas are resized to this size
batch_size_train = 4
batch_size_test = 1
learning_rate = 0.1
num_epoch = 10

In [ ]:
model = models.efficientnet_b1(weights="DEFAULT")

In [ ]:
num_params = sum(param.numel() for param in model.parameters())
print(f"Total number of parameters in model: {num_params}")

In [ ]:
# printing the model
model

In [ ]:
# freeze all layers for fine tuning (not doing this takes it very long to train)
for param in model.parameters():
    param.requires_grad = False

# number of inputs in last layer
num_ftrs = model.classifier[1].in_features

model.classifier[1] = nn.Linear(num_ftrs, num_out_ftrs)

# printing the last layer : classifier
model.classifier

model = model.to(device)

### Loading dataset in torch dataloader

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import time

In [ ]:
transform = transforms.Compose(
    [
        transforms.Resize((image_input_size,image_input_size)),
        transforms.ToTensor()
    ]
)

In [ ]:
class CUBDataset(Dataset):
    def __init__(self, image_id, id_to_url, image_class_labels, transform):
        self.x = [id_to_url[x] for x in image_id]
        self.y = [image_class_labels[x] for x in image_id]
        self.n_samples = len(image_id)
        self.transform = transform
    
    def __getitem__(self, index):
        img = Image.open(self.x[index])
        img = self.transform(img).expand(3,-1,-1)
        return img, self.y[index]-1
    
    def __len__(self):
        return self.n_samples

In [ ]:
dataset_train = CUBDataset(
    image_id=image_train,
    id_to_url=image,
    image_class_labels=image_class_labels,
    transform=transform
)
dataset_test = CUBDataset(
    image_id=image_test,
    id_to_url=image,
    image_class_labels=image_class_labels,
    transform=transform
)

In [ ]:
dataloader_train = DataLoader(
    dataset=dataset_train,
    batch_size=batch_size_train,
    shuffle=True
)
dataloader_test = DataLoader(
    dataset=dataset_test,
    batch_size=batch_size_test,
    shuffle=False
)

In [ ]:
print(len(dataloader_train))
print(len(dataloader_test))

In [ ]:
# dataiter = iter(dataloader_train)
# j = 1
# for i in dataloader_train:
#     features, labels = next(dataiter)
#     # if features.shape != torch.Size([1,3,244,244]):
#     features = features.expand(-1,3,244,244).numpy()
#     print(features.shape)
#     plt.imshow(features.squeeze().transpose(1,2,0))
#     plt.show()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=learning_rate
)

In [ ]:
n_total_steps = len(dataloader_train)
learning_rate = 0.001

print("Training Started")
time_start = time.time()
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(dataloader_train):
        
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        with torch.no_grad():
            output_one_hot = torch.softmax(outputs, dim=1).argmax(dim=1)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%300 == 0 or i==1498: 
            print(f"Epoch {epoch+1}/{num_epoch}, Step {i+1}/{n_total_steps}, Loss {loss.item():.4f}")
            print(f"\tModel {output_one_hot.tolist()} Acutal {labels.tolist()} Difference {(output_one_hot-labels).tolist()}")
            # print()
    print()
time_elapsed = time.time() - time_start
print(f"Training Finished in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")

In [ ]:
# PATH = "num_epoch_2_all_grad_on.pth"
# torch.save(model.state_dict(), PATH)

In [ ]:
with torch.no_grad():
    n_total_steps = len(dataloader_test)
    n_correct = 0
    n_samples = 0
    for i, (images, labels) in enumerate(dataloader_test):
        
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        _, predictions = torch.max(outputs, 1)
        print(predictions, labels)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
    acc = 100.0 * n_correct / n_samples
    print(f"accuracy = {acc}")